In [1]:
import pandas as pd
import sys, os, gc
sys.path.append("/home/fvalle/phd/master_thesis/hsbm/")

In [2]:
#GTEx
df = pd.read_csv('https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz', skiprows=2, compression='gzip', sep='\t')
df['ensg'] = [x[:15] for x in df['Name']]
df.set_index('Name', inplace=True)
df.set_index(['ensg'],inplace=True)
df=df.drop(['Description'],1)
genelist=pd.read_csv("https://stephenslab.github.io/count-clustering/project/utilities/gene_names_all_gtex.txt", header=None).values.ravel()
df = df[df.index.isin(genelist)]
df_file = pd.read_csv("https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt", sep='\t').loc[:,['SAMPID','SMTS', 'SMTSD']]
df_file.set_index('SAMPID', inplace=True)

In [3]:
df_file.groupby('SMTS').count().sort_values('SMTSD',ascending=False)

,SMTSD
SMTS,
Blood,3480
Brain,3326
Skin,2014
Esophagus,1582
Blood Vessel,1473
Adipose Tissue,1327
Heart,1141
Muscle,1132
Lung,867


In [ ]:
df_ub = df.reindex(columns=df_file[df_file['SMTS'].isin(['Colon', 'Ovary', 'Liver'])].index).dropna(how='all', axis=1)

In [ ]:
import scanpy as sc

In [ ]:
adata = sc.AnnData(X=df_ub.transpose(), obs=df_file.reindex(index=df_ub.columns))

In [ ]:
sc.pp.log1p(adata, copy=False)

In [ ]:
sc.pp.highly_variable_genes(adata)

In [ ]:
sc.pl.highly_variable_genes(adata, save="hvg.svg")

In [ ]:
hvg = adata.var[adata.var['highly_variable']==True].index
samples = adata.obs.index

In [ ]:
df_hsbm = df.reindex(index=hvg, columns = samples).applymap(lambda x: pd.np.log2(x+1)*1e2)

In [ ]:
gc.collect()

In [ ]:
from sbmtm import sbmtm

In [ ]:
model = sbmtm()

In [ ]:
model.load_graph("graph.xml.gz")

In [ ]:
model.make_graph_from_BoW_df(df_hsbm)
model.save_graph("graph.xml.gz")

In [ ]:
model.fit(verbose=True, parallel=True)

In [ ]:
model.save_data()

In [ ]:
df_hsbm.round().loc[df_hsbm.index[10],:].hist()